In [1]:
import pandas as pd
import torch

In [2]:
## load data.csv
c_names=["feature1_t0","feature2_t0","feature3_t0","feature1_t1","feature2_t1","feature3_t1","feature1_t2","feature2_t2","feature3_t2","score_t0","score_t1","score_t2","score","label"]
data = pd.read_csv('data.csv',names=c_names,skiprows=1)
data.head()

,feature1_t0,feature2_t0,feature3_t0,feature1_t1,feature2_t1,feature3_t1,feature1_t2,feature2_t2,feature3_t2,score_t0,score_t1,score_t2,score,label
0,1.275952,9.548464,20.857206,-1.640917,9.879204,19.796847,1.286034,7.853225,20.916851,-12.885027,-13.454494,-10.525529,-13.805164,1.0
1,2.209974,11.657870,18.994037,-1.583167,11.185200,20.050245,-0.040625,10.566484,19.212929,-11.824399,-13.999659,-11.552003,-10.581159,1.0
2,-0.928405,11.057740,22.520939,1.421343,11.178265,20.241026,-0.029118,10.790355,20.340290,-11.021825,-12.921086,-14.071246,-10.887280,1.0
3,0.583430,9.428129,19.557302,0.102584,9.106648,14.846111,-1.093920,10.778970,20.884420,-15.485350,-15.343734,-12.313570,-14.453414,0.0
4,0.142398,9.179987,20.252988,-1.483179,9.365358,19.794396,1.470028,12.056651,20.820639,-13.315184,-13.604042,-14.272901,-15.524159,0.0


In [3]:
## Split data
X = data.iloc[:,:-2]
y = data.iloc[:,-1]

# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [4]:
X_train = torch.tensor(X_train.to_numpy()).float()
X_test = torch.tensor(X_test.to_numpy()).float()

y_train = torch.tensor(y_train.to_numpy()).long()
y_test = torch.tensor(y_test.to_numpy()).long()

In [5]:
mean_train = X_train.mean(dim=0, keepdim=True)
std_train = X_train.std(dim=0, keepdim=True)

X_train = (X_train - mean_train)/std_train
X_test = (X_test - mean_train)/std_train

In [6]:
import argparse
import os

import networkx as nx

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, RandomSampler, random_split, TensorDataset
import torchvision.datasets as datasets
from torchvision.models import resnet18
from torchvision.models.vision_transformer import VisionTransformer, ViT_B_16_Weights, vit_b_16, _vision_transformer
import torchvision.transforms as transforms

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import Callback, EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.strategies.ddp import DDPStrategy

import wandb


LR = 3e-4 # learning rate
BS = 64 # batch size


train_set = TensorDataset(X_train, y_train)
test_set = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_set, batch_size=BS, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BS)

In [7]:
from models import BaseModule
mlp = nn.Sequential(
    nn.Linear(12, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
)
model = BaseModule(mlp, num_classes=2, lr=LR)

In [8]:
# ckpt_best = ModelCheckpoint(
#     dirpath='checkpoints',
#     filename='ckpt',
#     # monitor='val_loss',
#     mode='min'
# )

# lr_monitor = LearningRateMonitor(logging_interval='epoch')

# callbacks = [ckpt_best, lr_monitor]

trainer = Trainer(
    max_epochs=30,
    # callbacks=callbacks,
    accelerator='gpu',
    devices=[0],
    # strategy=DDPStrategy(find_unused_parameters=False),
    # logger=WandbLogger(),
    # log_every_n_steps=10
    default_root_dir='checkpoints'
)

# train
trainer.fit(model, train_loader)

# test
trainer.test(dataloaders=test_loader, ckpt_path='best')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type       | Params
-----------------------------------------
0 | model     | Sequential | 1.9 K 
1 | train_acc | Accuracy   | 0     
2 | test_acc  | Accuracy   | 0     
-----------------------------------------
1.9 K     Trainable params
0         Non-trainable params
1.9 K     Total params
0.008     Total estimated model params size (MB)
/home/tommaso/miniconda3/envs/saint_env/lib/python3.8/site-packages/pytorch_lightning/tr

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at checkpoints/lightning_logs/version_16/checkpoints/epoch=29-step=1770.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at checkpoints/lightning_logs/version_16/checkpoints/epoch=29-step=1770.ckpt
/home/tommaso/miniconda3/envs/saint_env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is 

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.672340452671051
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.672340452671051}]